In [1]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing in

In [2]:
!pip install -U peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 7.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: peft
    Found existing installation: peft 0.14.0
    Uninstalling peft-0.14.0:
      Successfully uninstalled peft-0.14.0


In [3]:
import torch
import os
import random
import pandas as pd
import numpy as np
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import BlipProcessor, BlipForQuestionAnswering
import bert_score
import transformers
from peft import PeftModel, PeftConfig
from PIL import Image

2025-05-18 08:41:25.346931: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747557685.548560      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747557685.606418      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
def process_dataset(input_csv_path, image_base_dir=None):
    output_csv = 'vqa_dataset_processed.csv'
    df = pd.read_csv(input_csv_path)
    
    df = df.iloc[20000:]
    
    print(f"Processing dataset with {df.shape[0]} entries after excluding the first 20,000 samples")
    processed_data = []
    product_type_questions = [
        "What is the product type?",
        "Can you identify the type of product?",
        "What kind of product is this?",
        "Identify the product category.",
        "What sort of item is this?",
        "Could you tell me the product type?",
        "What's the category of this product?"
    ]
    
    for _, row in df.iterrows():
        image_path = row['path']
        if image_base_dir:
            full_image_path = os.path.join(image_base_dir, image_path)
        else:
            full_image_path = image_path
        qa_pairs = [(row[f'q{i}'], row[f'a{i}'].lower()) for i in range(1, 5)
                    if pd.notna(row[f'q{i}']) and pd.notna(row[f'a{i}'])]
        if pd.notna(row['product_type']):
            product_type_question = random.choice(product_type_questions)
            product_type_answer = row['product_type'].lower().replace('_', ' ')
            qa_pairs.append((product_type_question, product_type_answer))
        description = row['vqa_description']
        for question, answer in qa_pairs:
            processed_data.append({
                'image_path': full_image_path,
                'description': description,
                'question': question,
                'answer': answer
            })
    processed_df = pd.DataFrame(processed_data)
    
    processed_df.to_csv(output_csv, index=False)
    print(f"Saved {len(processed_df)} QA pairs to {output_csv}")

In [10]:
class VQADataset(Dataset):
    def __init__(self, csv_path):
        self.df = pd.read_csv(csv_path)
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        try:
            image = Image.open(row['image_path']).convert('RGB')
        except:
            image = Image.new('RGB', (224, 224), color='white')
        return image, row['question'], row['answer'].lower().strip(), row['image_path']

In [11]:
def collate(batch):
    images, questions, answers, img_paths = zip(*batch)
    return list(images), list(questions), list(answers), list(img_paths)

In [12]:
def print_parameter_count(model):
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Parameters : {total_params:,}")

In [13]:
def create_dataloader(dataset_path, batch_size=16, num_workers=4):
    
    dataset = VQADataset(dataset_path)
    
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,  
        shuffle=False,
        num_workers=num_workers, 
        pin_memory=True,
        collate_fn=collate
    )
    
    return dataloader

In [14]:
def load_model_and_processor(peft_model_id, device):
    config = PeftConfig.from_pretrained(peft_model_id)
    # Load base model
    base_model = BlipForQuestionAnswering.from_pretrained(config.base_model_name_or_path)
    # Attach LoRA adapters
    model = PeftModel.from_pretrained(base_model, peft_model_id)
    # Load processor
    processor = BlipProcessor.from_pretrained(config.base_model_name_or_path,use_fast=True)
    model = model.to(device)
    return processor, model

In [15]:
def evaluate(model, processor, dataloader, device):
   
    img_paths, questions, predictions, references = [], [], [], []
    model.eval()
    with torch.inference_mode():
        for images, ques, answers, batch_img_paths in tqdm(dataloader):
            inputs = processor(images=images, text=ques, return_tensors="pt", padding=True, truncation=True)
            
            inputs = {k: v.to(device) for k, v in inputs.items()}
            
            generated_ids = model.generate(**inputs)
            
            generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)
            
            batch_preds = [t.split("Answer:")[-1].strip().lower() if "Answer:" in t else t.strip().lower() 
                          for t in generated_texts]

            img_paths.extend(batch_img_paths)
            questions.extend(ques)
            predictions.extend(batch_preds)
            references.extend(answers)
    
    return img_paths, questions, predictions, references

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [17]:
input_csv_path = "/kaggle/input/abo-metadata/vqa_qa_cleaned.csv"
image_base_dir = "/kaggle/input/dataset-amazon-object/images"
processed_dataset_path = "/kaggle/working/vqa_dataset_processed.csv"
peft_model_id = "/kaggle/input/final-checkpoint/pytorch/default/1/checkpoint-23370"

In [18]:
process_dataset(input_csv_path, image_base_dir)

Processing dataset with 8974 entries after excluding the first 20,000 samples
Saved 40012 QA pairs to vqa_dataset_processed.csv


In [19]:
df = pd.read_csv(processed_dataset_path)

In [20]:
df.tail()

,image_path,description,question,answer
40007,/kaggle/input/dataset-amazon-object/images/b3/...,the image features a black storage box with wh...,Can you identify the type of product?,storage box
40008,/kaggle/input/dataset-amazon-object/images/4f/...,the image shows a silver pressure cooker with ...,What is the color of the pressure cooker's han...,black
40009,/kaggle/input/dataset-amazon-object/images/4f/...,the image shows a silver pressure cooker with ...,What is the color of the background in the image?,white
40010,/kaggle/input/dataset-amazon-object/images/4f/...,the image shows a silver pressure cooker with ...,Is the lid open or closed in the image?,closed
40011,/kaggle/input/dataset-amazon-object/images/4f/...,the image shows a silver pressure cooker with ...,Can you identify the type of product?,pressure cooker


In [21]:
dataloader = create_dataloader(processed_dataset_path)

In [22]:
processor, model = load_model_and_processor(peft_model_id, device)

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [23]:
print_parameter_count(model)

Parameters : 385,852,220


In [24]:
img_paths, questions, predictions, references = evaluate(model, processor, dataloader, device)

100%|██████████| 2501/2501 [20:59<00:00,  1.99it/s]


In [25]:
transformers.logging.set_verbosity_error()

In [26]:
# Accuracy calculation
exact_matches = [p == r for p, r in zip(predictions, references)]
accuracy = sum(exact_matches) / len(exact_matches)

In [27]:
# F1 Score calculation
pred_tokens = [set(p.split()) for p in predictions]
ref_tokens = [set(r.split()) for r in references]

f1s = []
for p, r in zip(pred_tokens, ref_tokens):
    precision = len(p.intersection(r))/len(p) if len(p) > 0 else 0.0
    recall = len(p.intersection(r))/len(r) if len(r) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    f1s.append(f1)

avg_f1 = sum(f1s) / len(f1s)

In [28]:
# BERTScore calculation
bert_device = device.type
P, R, F1 = bert_score.score(predictions, references, lang="en", 
                           batch_size=16 if torch.cuda.is_available() else 8, 
                           device=bert_device)
bert_score_val = F1.mean().item()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [29]:
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {avg_f1:.4f}")
print(f"BERTScore: {bert_score_val:.4f}")

Accuracy: 0.5118
F1 Score: 0.5184
BERTScore: 0.9747


In [30]:
result = pd.DataFrame({
        'image_path': img_paths,
        'question': questions,
        'prediction': predictions,
        'reference': references
    })

In [31]:
result.to_csv('blip-vqa-base_fine_tuned-v5.csv', index=False)